In [1]:
from keras.datasets import fashion_mnist
import numpy as np
from tensorflow.keras.utils import to_categorical
import wandb
from sklearn.metrics import accuracy_score

In [2]:
# Loading the data
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

# Normalize the data
x_train = x_train.reshape(-1, 784) / 255.0
x_test = x_test.reshape(-1, 784) / 255.0

# One-hot encode labels
y_train, y_test = to_categorical(y_train), to_categorical(y_test)

# Split validation data
val_split = int(0.1 * x_train.shape[0])

# Create validation data
x_val, y_val = x_train[:val_split], y_train[:val_split]
x_train, y_train = x_train[val_split:], y_train[val_split:]

x_train = x_train.T
y_train = y_train.T
x_val = x_val.T
y_val = y_val.T
x_test = x_test.T
y_test = y_test.T

x_train.shape, y_train.shape

((784, 54000), (10, 54000))

class FeedforwardNeuralNetwork:
    def __init__(self, input_size, hidden_layers, output_size):
        """
        Initialize the neural network.
        :param input_size: Number of input features.
        :param hidden_layers: List containing the number of neurons in each hidden layer.
        :param output_size: Number of output classes.
        """
        self.layers = []
        self.weights = []
        self.biases = []
        self.velocities_w = []  # For momentum-based optimizers
        self.velocities_b = []
        self.squared_w = []  # For RMSprop/Adam
        self.squared_b = []
        self.moments_w = []  # For Nadam
        self.moments_b = []

        # Input layer to first hidden layer
        prev_size = input_size
        for layer_size in hidden_layers:
            self.weights.append(np.random.randn(layer_size, prev_size))
            self.biases.append(np.zeros((layer_size, 1)))
            self.velocities_w.append(np.zeros((layer_size, prev_size)))
            self.velocities_b.append(np.zeros((layer_size, 1)))
            self.squared_w.append(np.zeros((layer_size, prev_size)))
            self.squared_b.append(np.zeros((layer_size, 1)))
            self.moments_w.append(np.zeros((layer_size, prev_size)))
            self.moments_b.append(np.zeros((layer_size, 1)))
            prev_size = layer_size

        # Hidden layers to output layer
        self.weights.append(np.random.randn(output_size, prev_size))
        self.biases.append(np.zeros((output_size, 1)))
        self.velocities_w.append(np.zeros((output_size, prev_size)))
        self.velocities_b.append(np.zeros((output_size, 1)))
        self.squared_w.append(np.zeros((output_size, prev_size)))
        self.squared_b.append(np.zeros((output_size, 1)))
        self.moments_w.append(np.zeros((output_size, prev_size)))
        self.moments_b.append(np.zeros((output_size, 1)))
    
    def train(self, x_train, y_train, x_val, y_val, epochs, learning_rate, optimizer='sgd', batch_size=32):

        for epoch in range(epochs):
            indices = np.arange(x_train.shape[1])
            np.random.shuffle(indices)
            x_train, y_train = x_train[:,indices], y_train[:,indices]
            
            for i in range(0, x_train.shape[1], batch_size):
                x_batch = x_train[:,i:i+batch_size]
                y_batch = y_train[:,i:i+batch_size]
                self.forward(x_batch)
                self.backpropagation(x_batch, y_batch, learning_rate, optimizer)
        
            train_pred, val_pred = self.forward(x_train), self.forward(x_val)
            train_loss, val_loss = self.cross_entropy(train_pred, y_train), self.cross_entropy(val_pred, y_val)
            train_cls, val_cls = np.argmax(train_pred, axis = 0), np.argmax(val_pred, axis = 0)
            train_acc, val_acc = accuracy_score(np.argmax(y_train, axis=0), train_cls), accuracy_score(np.argmax(y_val, axis=0), val_cls)

            print(f"Epoch {epoch}: Training Loss = {train_loss:.4f} Validation Loss = {val_loss:.4f}")
            print(f"Training Accuracy = {train_acc:.4f} Validation Accuracy = {val_acc:.4f}")

            wandb.log({
                "epoch" : epoch, 
                "train_loss" : train_loss, 
                "val_loss" : val_loss, 
                "train_acc" : train_acc, 
                "val_acc" : val_acc
            })
        
        wandb.log({"Accuracy" : val_acc})
        
    
    @staticmethod
    def cross_entropy(y_pred, y_true) :
        eps = 1e-8 
        y_pred = np.clip(y_pred, eps, 1 - eps)
        return -np.sum(y_true * np.log(y_pred)) / y_true.shape[1]

    def sigmoid(self, x):
        """Sigmoid activation function."""
        return 1 / (1 + np.exp(-x))

    def sigmoid_derivative(self, x):
        """Derivative of the sigmoid function."""
        return x * (1 - x)

    def softmax(self, x):
        """Softmax activation function."""
        exp_x = np.exp(x - np.max(x, axis=0, keepdims=True))
        return exp_x / np.sum(exp_x, axis=0, keepdims=True)

    def forward(self, x):
        """
        Perform a forward pass through the network.
        :param x: Input data.
        :return: Output probabilities.
        """
        self.layers = [x]
        for w, b in zip(self.weights[:-1], self.biases[:-1]):
            x = self.sigmoid(np.dot(w,x) + b)
            # x = self.sigmoid(np.dot(x, w) + b)
            self.layers.append(x)

        # Output layer with softmax
        output = self.softmax(np.dot(self.weights[-1], x) + self.biases[-1])
        self.layers.append(output)
        return output

    def backpropagation(self, x, y, learning_rate, optimizer='sgd', beta1=0.9, beta2=0.999, epsilon=1e-8):
        m = y.shape[1]  # Number of samples
        deltas = [self.layers[-1] - y]  # Output layer error

        # Backpropagate errors for hidden layers
        for i in range(len(self.weights) - 1, 0, -1):
            deltas.append(self.weights[i].T.dot(deltas[-1]) * self.sigmoid_derivative(self.layers[i]))
        deltas.reverse()

        # Update weights and biases based on optimizer
        for i in range(len(self.weights)):
            dw = (self.layers[i].dot(deltas[i].T)).T / m
            db = np.mean(deltas[i], axis=1, keepdims=True)

            if optimizer == 'sgd':
                self.weights[i] -= learning_rate * dw
                self.biases[i] -= learning_rate * db
            
            elif optimizer == 'momentum':
                self.velocities_w[i] = beta1 * self.velocities_w[i] - learning_rate * dw
                self.velocities_b[i] = beta1 * self.velocities_b[i] - learning_rate * db
                self.weights[i] += self.velocities_w[i]
                self.biases[i] += self.velocities_b[i]
            
            elif optimizer == 'nag':
                prev_w = self.velocities_w[i]
                prev_b = self.velocities_b[i]
                self.velocities_w[i] = beta1 * prev_w - learning_rate * dw
                self.velocities_b[i] = beta1 * prev_b - learning_rate * db
                self.weights[i] += -beta1 * prev_w + (1 + beta1) * self.velocities_w[i]
                self.biases[i] += -beta1 * prev_b + (1 + beta1) * self.velocities_b[i]
            
            elif optimizer == 'rmsprop':
                self.squared_w[i] = beta2 * self.squared_w[i] + (1 - beta2) * (dw ** 2)
                self.squared_b[i] = beta2 * self.squared_b[i] + (1 - beta2) * (db ** 2)
                self.weights[i] -= learning_rate * dw / (np.sqrt(self.squared_w[i]) + epsilon)
                self.biases[i] -= learning_rate * db / (np.sqrt(self.squared_b[i]) + epsilon)
            
            elif optimizer == 'adam' or optimizer == 'nadam':
                self.velocities_w[i] = beta1 * self.velocities_w[i] + (1 - beta1) * dw
                self.velocities_b[i] = beta1 * self.velocities_b[i] + (1 - beta1) * db
                self.squared_w[i] = beta2 * self.squared_w[i] + (1 - beta2) * (dw ** 2)
                self.squared_b[i] = beta2 * self.squared_b[i] + (1 - beta2) * (db ** 2)
                if optimizer == 'nadam':
                    dw = beta1 * self.velocities_w[i] + (1 - beta1) * dw
                    db = beta1 * self.velocities_b[i] + (1 - beta1) * db
                self.weights[i] -= learning_rate * dw / (np.sqrt(self.squared_w[i]) + epsilon)
                self.biases[i] -= learning_rate * db / (np.sqrt(self.squared_b[i]) + epsilon)

In [ ]:
sweep_config = {
    "name": "Feedforward Network - Hyper parameter search",
    "metric": {
        "name": "Accuracy",
        "goal": "maximize"
    },
    "method": "random",
    "parameters": {
        "num_epochs": {
            "values": [5, 10]
            },
        "num_hiddenLayers": {
            "values": [3, 4, 5]
            },
        "hiddenLayer_Size": {
            "values": [32, 64, 128]
            },
        "weightDecay": {
            "values": [0, 0.0005, 0.5]
            },
        "learningRate": {
            "values": [1e-3, 1e-4]
            },
        "optimizer": {
            "values": ["sgd", "momentum", "nesterov", "rmsprop", "adam", "nadam"]
            },
        "batchSize": {
            "values": [16, 32, 64]
            },
        "weightInit": {
            "values": ["random", "xavier"]
            },
        "activationFunc": {
            "values": ["tanh", "relu", "sigmoid"]
            }
    }
    
}

In [ ]:
def sweep_hyperParameters():
    default_config = {
        'num_epochs': 10,
        'num_hiddenLayers': 3,
        'hiddenLayer_Size': 40,
        'weightDecay': 0,
        'learningRate': 1e-3,
        'optimizer': 'Nesterov_Accelerated_GD',
        'batchSize': 32,
        'weightInit': 'xavier',
        'activationFunc': 'sigmoid'
    }
    
    wandb.init(project="assignment1", entity="da6401-assignments")
    wandb.init(config = default_config)

    config = wandb.config # To get the Hyper Parameters from sweep_config
    
    # Parameters from wandb.config
    num_epochs = config.num_epochs
    num_hiddenLayers = config.num_hiddenLayers
    hiddenLayer_size = config.hiddenLayer_Size
    weightDecay = config.weightDecay
    learningRate = config.learningRate
    optimizer = config.optimizer
    batchSize = config.batchSize
    weightInit = config.weightInit
    activationFunc = config.activationFunc
    
    # Name
    run_name = f"LR: {learningRate}, AC: {activationFunc}, BS: {batchSize}, Optim: {optimizer}, WI: {weightInit}, WD: {weightDecay}, No_HL: {num_hiddenLayers}, HS: {hiddenLayer_size}, EP: {num_epochs}"
    print("Sweep Name: ",run_name)
    
    model = FeedforwardNeuralNetwork(x_train.shape[0], [hiddenLayer_size] * num_hiddenLayers, y_train.shape[0])
    
    wandb.run.name = run_name
    
    model.train(x_train, y_train, x_val, y_val, num_epochs, learningRate, optimizer, batchSize)

    wandb.run.save()

    return model 

In [ ]:
sweep_ID = wandb.sweep(sweep_config, entity="da6401-assignments", project="assignment1")
wandb.agent(sweep_ID, sweep_hyperParameters, count=5)

wandb.finish()

In [ ]:
api = wandb.Api()

runs = api.runs("da6401-assignments/assignment1")

best_run = min(runs, key=lambda run: run.summary.get("val_loss", float("inf")))

In [21]:
print(f"Best run name: {best_run.name}. \nValidation Loss: {best_run.summary.get('val_loss')}")

Best run name: LR: 0.001, AC: relu, BS: 16, Optim: nadam, WI: xavier, WD: 0, No_HL: 4, HS: 64. 
Validation Loss: 0.3719278325466944


In [25]:
import re

pairs = re.findall(r'(\w+): ([^,]+)', best_run.name)

param_dict = {key: value.strip() for key, value in pairs}

param_dict = {
    key: int(value) if value.isdigit() else float(value) if value.replace('.', '', 1).isdigit() else value
    for key, value in param_dict.items()
}

print(param_dict)

{'LR': 0.001, 'AC': 'relu', 'BS': 16, 'Optim': 'nadam', 'WI': 'xavier', 'WD': 0, 'No_HL': 4, 'HS': 64}


In [27]:
best_model = FeedforwardNeuralNetwork(input_size= 784, hidden_layers= [param_dict['HS']] * param_dict['No_HL'], output_size= 10)

In [ ]:
best_model.train(x_train, y_train, x_val, y_val, 
                    epochs= param_dict['EP'], 
                    learning_rate= param_dict['LR'], 
                    optimizer= param_dict['optim'], 
                    batch_size= param_dict['BS'])